In [21]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

import model as modellib
from config import Config
import utils


## Load dataset 

In [10]:
import csv
import dataset

# Load class map - these tables map the original TACO classes to your desired class system
# and allow you to discard classes that you don't want to include.
class_map = {}
with open("./taco_config/map_1.csv") as csvfile:
    reader = csv.reader(csvfile)
    class_map = {row[0]:row[1] for row in reader}

# Load full dataset or a subset
TACO_DIR = "../../../data/tacodataset/"
round = None # Split number: If None, loads full dataset else if int > 0 selects split no 
subset = "train" # Used only when round !=None, Options: ('train','val','test') to select respective subset
dataset = dataset.Taco()
taco = dataset.load_taco(TACO_DIR, round, subset, class_map=class_map, return_taco=True)

# Must call before using the dataset
dataset.prepare()

print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))



creating index...
index created!
Number of images used: 1500
Class Count: 2
  0. BG                                                
  1. Litter                                            


## Check class balance

In [15]:
import seaborn as sns
import pandas as pd

# Count annotations
cat_histogram = np.zeros(dataset.num_classes,dtype=int)
class_names = []
for i, info in enumerate(dataset.class_info):
    ann_per_cat = taco.getAnnIds(catIds=i, iscrowd=None)
    cat_histogram[i] = len(ann_per_cat)
    class_names.append(info['name'])

for i in range(2):
    print(f"{class_names[i]}: {cat_histogram[i]}")


# As we are doing object segmentation and not image classification, we can work with only positive examples. 
# The background of the image will be serving as the negative example. 



BG: 0
Litter: 4784


In [20]:


class TacoTestConfig(Config):
    NAME = "taco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.3
    NUM_CLASSES = dataset.num_classes
#     IMAGE_MAX_DIM = 1024
#     IMAGE_MIN_DIM = 1024
#     IMAGE_RESIZE_MODE = "square"
config = TacoTestConfig()



In [22]:
from random import seed
from random import randint
import skimage.io
import skimage.transform

seed(1)

# Load random image
image_id = np.random.choice(len(dataset.image_ids))
image_ori = dataset.load_image(image_id)
masks_ori, _ = dataset.load_mask(image_id)

image_dtype = image_ori.dtype
nr_annotations = np.shape(masks_ori)[-1]

bboxes = utils.extract_bboxes(masks_ori)

# Initialize the matplotlib figure
fig, ax = plt.subplots(nr_annotations+1, 1, figsize=(10,10*(nr_annotations+1)))
ax[0].imshow(image_ori)

i=0
for bbox_id in range(len(bboxes)):
    
    image = image_ori
    masks = masks_ori
    
    bboxes_cpy = bboxes
    y1, x1, y2, x2 = bboxes_cpy[bbox_id]
    h, w = image.shape[:2]
    
    bbox_width = x2-x1
    bbox_height = y2-y1
    
    img_max_dim = max(h,w)
    bbox_max_dim = max(bbox_width,bbox_height)
    
    print('Image original shape:',image.shape)
    print('Bbox original shape:',y1, x1, y2, x2)
    
    bbox_max_dim_threshold_4_scaling = config.IMAGE_MAX_DIM*0.8
    
    # If bbox is big enough or too big, downsize full image
    if bbox_max_dim > bbox_max_dim_threshold_4_scaling:
        
        # Rescale
        downscale_at_least = min(1.,config.IMAGE_MAX_DIM/bbox_max_dim)
        downscale_min = config.IMAGE_MAX_DIM/img_max_dim
        
        scale = random.random()*(downscale_at_least-downscale_min)+downscale_min
        
        print('Downscaling image by',scale)
        print("Scale interval:", downscale_min, downscale_at_least)
        
        # Actually scale Image   
        image = skimage.transform.resize(image, (np.round(h * scale), np.round(w * scale)),order=1,
                                         mode="constant", preserve_range=True)
    
        h, w = image.shape[:2]
        img_max_dim = max(h,w)
    
        # Padding
        top_pad = (img_max_dim - h) // 2
        bottom_pad = img_max_dim - h - top_pad
        left_pad = (img_max_dim - w) // 2
        right_pad = img_max_dim - w - left_pad
        padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
        image = np.pad(image, padding, mode='constant', constant_values=0).astype(image_dtype)
        window = (top_pad, left_pad, h + top_pad, w + left_pad)
        
        # Adjust mask and other vars
        masks = utils.resize_mask(masks, scale, padding)
        bboxes_cpy = utils.extract_bboxes(masks)
        y1, x1, y2, x2 = bboxes_cpy[bbox_id]
        h, w = image.shape[:2]
        
        print('Image resized shape:',image.shape)
    
    # Select crop around target annotation
    x0_min = max(x2-config.IMAGE_MAX_DIM,0) 
    x0_max = min(x1,w-config.IMAGE_MAX_DIM)
    x0 = randint(x0_min, x0_max)
    y0_min = max(y2-config.IMAGE_MAX_DIM,0) 
    y0_max = min(y1,h-config.IMAGE_MAX_DIM)
    y0 = randint(y0_min, y0_max)
    
    if padding:
        max_dim = config.IMAGE_MAX_DIM
        window = (max(top_pad,y0)-y0, max(left_pad,x0)-x0, min(window[2],y0+max_dim)-y0, min(window[3],x0+max_dim)-x0)
        print(window)
    
    # Crop
    crop = (y0, x0, config.IMAGE_MAX_DIM, config.IMAGE_MAX_DIM)
    image = image[y0:y0 + config.IMAGE_MAX_DIM, x0:x0 + config.IMAGE_MAX_DIM]
    masks = masks[y0:y0 + config.IMAGE_MAX_DIM, x0:x0 + config.IMAGE_MAX_DIM]
    ax[i+1].imshow(image)
    ax[i+1].axis('off')
    i+=1

Image original shape: (3264, 2448, 3)
Bbox original shape: 882 431 1998 1593
Downscaling image by 0.3899790465726482
Scale interval: 0.3137254901960784 0.8812392426850258
Image resized shape: (1273, 1273, 3)
(0, 0, 1024, 898)
Image original shape: (3264, 2448, 3)
Bbox original shape: 1398 1175 2117 1926
(0, 0, -199, -36)


In [23]:
import imgaug as ia
from imgaug import augmenters as iaa
import imageio

#ia.seed(1)
nr_augmentations = 10

# Load random image
# image_id = np.random.choice(len(dataset.image_ids))
# image = dataset.load_image(image_id)
# masks, _ = dataset.load_mask(image_id)

# Define our augmentation pipeline.
seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(scale=0.01 * 255, name="AWGN"),
        iaa.GaussianBlur(sigma=(0.0, 3.0), name="Blur"),
        # iaa.Dropout([0.0, 0.05], name='Dropout'), # drop 0-5% of all pixels
        iaa.Fliplr(0.5),
        iaa.Add((-20, 20),name="Add"),
        iaa.Multiply((0.8, 1.2), name="Multiply"),
#       iaa.Affine(scale=(0.8, 1.1)),
#       iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
        iaa.Affine(rotate=(-45, 45)),  # rotate by -45 to 45 degrees
    ], random_order=True)

# Change activated augmentors for masks
def activator_masks(images, augmenter, parents, default):
    if augmenter.name in ["Blur", "AWGN", "Add","Multiply"]:
        return False
    else:
        # default value for all other augmenters
        return default
    
hooks_masks = ia.HooksImages(activator=activator_masks)

# Initialize the matplotlib figure
fig, aux = plt.subplots(ncols=2, nrows=nr_augmentations+1, figsize=(15,60))

aux[0,0].imshow(image)
aux[0,0].axis('off')
aux[0,1].imshow(masks.sum(axis=2)) # form an unique segmask from multiple masks
aux[0,1].axis('off')

print('This may take some time, as we are working with the original image size...')

# Augment images and masks
for i in range(nr_augmentations):
    seq_det = seq.to_deterministic()
    image_augmented = seq_det.augment_image(image)
    mask_augmented = seq_det.augment_image(masks.astype(np.uint8), hooks=hooks_masks)
    seg_mask = mask_augmented.sum(axis=2)

    aux[i+1,0].imshow(image_augmented)
    aux[i+1,0].axis('off')
    aux[i+1,1].imshow(seg_mask)
    aux[i+1,1].axis('off')
    
plt.subplots_adjust(wspace=0.2, hspace=0)

This may take some time, as we are working with the original image size...
